# Docker Basics

Docker is a platform that allows you to automate the deployment of applications inside lightweight, portable containers. This tutorial will guide you through the basics of Docker: installation, pulling and running containers, building Docker images, and basic container management.

Before we begin, let's check if Docker is installed and running on your machine.

## Checking Docker Installation

In [1]:
!docker --version

Docker version 28.4.0, build d8eb465


If Docker is installed, this will display the installed version. If not, please install Docker before proceeding. To avoid complex setup, it is recommended to install [Docker Desktop](https://docs.docker.com/desktop/).

## Pulling Your First Docker Image
Docker containers are built from Docker images. In this section, we'll pull a pre-built image from [Docker Hub](https://hub.docker.com/).

We'll start by pulling the official `hello-world` image.

In [2]:
!docker pull hello-world

Using default tag: latest
latest: Pulling from library/hello-world
Digest: sha256:d4aaab6242e0cace87e2ec17a2ed3d779d18fbfd03042ea58f2995626396a274
Status: Image is up to date for hello-world:latest
docker.io/library/hello-world:latest


On Linux: If you received a 'permission denied' error, try following the instructions at: https://www.digitalocean.com/community/questions/how-to-fix-docker-got-permission-denied-while-trying-to-connect-to-the-docker-daemon-socket in a terminal window.

## Running a Docker Container
Now that we have the `hello-world` image, let's run it as a container.

In [3]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



When you run this, Docker will start a container using the `hello-world` image. It will print a message and then exit. The message demonstrates that your Docker installation is working properly.

## Listing Running and Stopped Containers
Docker keeps a history of the containers you've run. You can list them using the following command:

In [4]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND                  CREATED         STATUS                     PORTS     NAMES
adf783395cd4   hello-world   "/hello"                 4 seconds ago   Exited (0) 3 seconds ago             gracious_feynman
cb29c396c062   influxdb      "/entrypoint.sh infl…"   5 weeks ago     Exited (0) 5 weeks ago               influxdb-server
37692bca36dd   rabbitmq      "docker-entrypoint.s…"   5 weeks ago     Exited (0) 5 weeks ago               rabbitmq-server


This command will display all containers, including stopped ones, along with their ID, name, status, and other details.

## Removing a Docker Container
To clean up, let's remove the `hello-world` container we just created.

First, find the container ID or name using `docker ps -a`, and then run, replace the ID in the next command and run:

In [6]:
!docker rm adf783395cd4

adf783395cd4


Let's verify that the container was removed.

In [7]:
!docker ps -a

CONTAINER ID   IMAGE      COMMAND                  CREATED       STATUS                   PORTS     NAMES
cb29c396c062   influxdb   "/entrypoint.sh infl…"   5 weeks ago   Exited (0) 5 weeks ago             influxdb-server
37692bca36dd   rabbitmq   "docker-entrypoint.s…"   5 weeks ago   Exited (0) 5 weeks ago             rabbitmq-server


## Using Shared Volumes in Docker

Shared volumes allow you to share files between your host machine and the Docker container. This is useful for editing files on your local machine while having those changes immediately reflected in the container.

We'll create a shared volume between a folder on your host machine and the container running a simple Python script.

###
First, let's create a directory on your host machine where we can store a Python script.

In [8]:
# Create directory for the shared volume in python
import os
print("Creating directory in folder: " + os.getcwd())
os.makedirs("shared_volume", exist_ok=True)

Creating directory in folder: /home/sunny/work/LOG6310E/bentley/IncubatorDTCourse/0-Pre-requisites


### Create a Python Script on Your Host

In [9]:
%%writefile ./shared_volume/hello_from_volume.py
print("Hello from the shared volume!")

Overwriting ./shared_volume/hello_from_volume.py


## Building a Docker Image
Now, let's move on to creating our own Docker image. We'll create a simple image that runs a Python script from the shared volume.

First, create a `Dockerfile`. This file contains instructions for building a Docker image. Here's an example:

In [10]:
%%writefile Dockerfile
# Use Python base image
FROM python:3.8-slim

# Set the working directory inside the container
WORKDIR /app

# Command to run a script from the volume (this script will be provided by the volume)
CMD ["python", "/app/hello_from_volume.py"]

Overwriting Dockerfile


Now, build the Docker image using the `docker build` command. The `-t` flag allows you to tag the image with a name:

In [11]:
!docker build -t my-python-app .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 279B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 279B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 279B                                       0.0s
 => [internal] load metadata for docker.

## Running the Custom Docker Image

We will now run the container, mounting the local directory (`./shared_volume`) into the container at the `/app` directory. This way, the Python script on your local machine will be accessible from inside the container.

Explanation:
- **`-v "{shared_vol_dir}:/app"`**: This option mounts the host directory (`{shared_vol_dir}`) to the container’s `/app` directory.
- **`--rm`**: Automatically removes the container once it exits.
- **`my-python-app`**: This is the image we built earlier.

In [12]:
shared_vol_dir = os.path.abspath("./shared_volume")
shared_vol_dir

'/home/sunny/work/LOG6310E/bentley/IncubatorDTCourse/0-Pre-requisites/shared_volume'

In [13]:
!docker run --rm -v "{shared_vol_dir}:/app" my-python-app

Hello from the shared volume!


You should see the output: `Hello from the shared volume!`.

## Editing the Script

Let’s modify the Python script from your local machine and re-run the container to see how changes are reflected:

In [14]:
%%writefile ./shared_volume/hello_from_volume.py
print("Hello again from the updated shared volume!")

Overwriting ./shared_volume/hello_from_volume.py


Run the container again:

In [15]:
!docker run --rm -v "{shared_vol_dir}:/app" my-python-app

Hello again from the updated shared volume!


The container will now output: `Hello again from the updated shared volume!`.


## Clean Up

To remove the image you created:

In [16]:
!docker rmi my-python-app

Untagged: my-python-app:latest
Deleted: sha256:a7f89b4b7527d821a3c21b79bb23b1f5be7134822221976346258ba970c49658


In [17]:
# Remove all stopped containers
!docker container prune -f

Deleted Containers:
cb29c396c062a40cbf084aea25e619724a8e4b5f8166a8a0fa90995afb18108c
37692bca36ddedfd88c476bb1613412433cccc86a8ecb178f9f40f5e9dfd7eb0

Total reclaimed space: 0B


In [18]:
!docker rm 4109a4ad88aa

Error response from daemon: No such container: 4109a4ad88aa


In [ ]:
# Create directory for the shared volume in python
import os
os.makedirs("shared_volume", exist_ok=True)